# Random Forest Experiments for Fall Prediction

This notebook implements three experiments from the research paper:
- **Experiment I**: Base RF model with linear and nonlinear variables separately
- **Experiment II**: RF with feature engineering (unsupervised selection + PCA)
- **Experiment III**: Combined model with elbow point analysis

## Setup and Data Loading

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve
from sklearn.feature_selection import VarianceThreshold
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

In [ ]:
# Load data
df = pd.read_csv('../data/combined_output.csv')
print(f"Dataset shape: {df.shape}")
print(f"\nClass distribution:\n{df['Faller'].value_counts()}")

# Check for missing values
print(f"\nMissing values per column:")
missing = df.isnull().sum()
print(missing[missing > 0])

# Handle missing values with median imputation
imputer = SimpleImputer(strategy='median')
feature_cols = [col for col in df.columns if col not in ['ID', 'Faller']]
df[feature_cols] = imputer.fit_transform(df[feature_cols])

print(f"\nAfter imputation - Missing values: {df.isnull().sum().sum()}")
print(f"\nFirst few rows:")
df.head()

## Define Linear and Nonlinear Variables

Based on the paper description:
- **Linear variables**: Mean values, basic measurements (timing means, velocity, RMS, anthropometry)
- **Nonlinear variables**: Variability measures (CV, sdTotal), MSE, RQA, HR (harmony/regularity)

In [ ]:
# Define linear variables (mean values and basic measurements)
linear_variables = [
    # Timing means
    'GCTime_mean', 'RSST_mean', 'LSST_mean', 'RSwT_mean', 'LSwT_mean', 'DST_mean', 'StepTime_mean',
    # RMS acceleration
    'RMS_AP', 'RMS_V', 'RMS_ML',
    # RMSR (residual)
    'RMSR_AP', 'RMSR_ML', 'RMSR_V',
    # Velocity measures
    'Velocity', 'Time2FirstQuartile_Velocity', 'Time2Median_Velocity', 'Time2ThirdQuartile_Velocity',
    # Anthropometry
    'Age', 'Height (m)', 'Weight (lbs)'
]

# Define nonlinear variables (variability, complexity, dynamical measures)
nonlinear_variables = [
    # Variability measures (sdTotal and cv)
    'GCTime_sdTotal', 'GCTime_cv',
    'RSST_sdTotal', 'RSST_cv',
    'LSST_sdTotal', 'LSST_cv',
    'RSwT_sdTotal', 'RSwt_cv',
    'LSwT_sdTotal', 'LSwT_cv',
    'DST_sdTotal', 'DST_cv',
    'StepTime_sdTotal', 'StepTime_cv',
    # Harmony/Regularity
    'HR_AP', 'HR_ML', 'HR_V',
    # Multiscale Entropy (area and slope)
    'MSE_AP_area', 'MSE_ML_area', 'MSE_V_area', 'MSE_Res_area',
    'MSE_AP_slope', 'MSE_ML_slope', 'MSE_V_slope', 'MSE_Res_slope',
    # Recurrence Quantification Analysis
    'RQA_AP_Rec', 'RQA_AP_Det', 'RQA_AP_Ent', 'RQA_AP_MaxLine',
    'RQA_ML_Rec', 'RQA_ML_Det', 'RQA_ML_Ent', 'RQA_ML_MaxLine',
    'RQA_V_Rec', 'RQA_V_Det', 'RQA_V_Ent', 'RQA_V_MaxLine',
    'RQA_Res_Rec', 'RQA_Res_Det', 'RQA_Res_Ent', 'RQA_Res_MaxLine'
]

print(f"Number of linear variables: {len(linear_variables)}")
print(f"Number of nonlinear variables: {len(nonlinear_variables)}")
print(f"Total: {len(linear_variables) + len(nonlinear_variables)}")

## Train-Test Split

The paper mentions 127 training participants and 44 test participants (total 171).

In [ ]:
# Create train-test split
# Use first 127 for training, last 44 for testing
train_df = df.iloc[:127].copy()
test_df = df.iloc[127:].copy()

print(f"Training set size: {len(train_df)}")
print(f"Test set size: {len(test_df)}")
print(f"\nTraining set class distribution:\n{train_df['Faller'].value_counts()}")
print(f"\nTest set class distribution:\n{test_df['Faller'].value_counts()}")

# Convert Faller to binary (F=1, NF=0)
train_df['Faller'] = (train_df['Faller'] == 'F').astype(int)
test_df['Faller'] = (test_df['Faller'] == 'F').astype(int)

## Helper Functions

In [ ]:
def calculate_metrics(y_true, y_pred, n_runs=10):
    """
    Calculate accuracy, sensitivity, and specificity with std error.
    For single run, returns metrics without std.
    """
    if n_runs == 1:
        cm = confusion_matrix(y_true, y_pred)
        tn, fp, fn, tp = cm.ravel()
        
        accuracy = (tp + tn) / (tp + tn + fp + fn)
        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        
        return {
            'accuracy': accuracy * 100,
            'sensitivity': sensitivity * 100,
            'specificity': specificity * 100
        }
    else:
        # For multiple runs, y_pred should be a list of predictions
        accuracies, sensitivities, specificities = [], [], []
        
        for pred in y_pred:
            cm = confusion_matrix(y_true, pred)
            tn, fp, fn, tp = cm.ravel()
            
            accuracies.append((tp + tn) / (tp + tn + fp + fn))
            sensitivities.append(tp / (tp + fn) if (tp + fn) > 0 else 0)
            specificities.append(tn / (tn + fp) if (tn + fp) > 0 else 0)
        
        return {
            'accuracy': np.mean(accuracies) * 100,
            'accuracy_std': np.std(accuracies, ddof=1) * 100,
            'sensitivity': np.mean(sensitivities) * 100,
            'sensitivity_std': np.std(sensitivities, ddof=1) * 100,
            'specificity': np.mean(specificities) * 100,
            'specificity_std': np.std(specificities, ddof=1) * 100
        }

def train_rf_model(X_train, y_train, X_test, n_trees=365, max_features=1, random_state=42, n_runs=10):
    """
    Train Random Forest model with specified parameters.
    Returns predictions for multiple runs if n_runs > 1.
    """
    if n_runs == 1:
        rf = RandomForestClassifier(
            n_estimators=n_trees,
            max_features=max_features,
            random_state=random_state,
            n_jobs=-1
        )
        rf.fit(X_train, y_train)
        return rf.predict(X_test), rf
    else:
        predictions = []
        models = []
        
        for seed in range(n_runs):
            rf = RandomForestClassifier(
                n_estimators=n_trees,
                max_features=max_features,
                random_state=random_state + seed,
                n_jobs=-1
            )
            rf.fit(X_train, y_train)
            predictions.append(rf.predict(X_test))
            models.append(rf)
        
        return predictions, models

def print_metrics(metrics, title):
    """Print metrics in a formatted way."""
    print(f"\n{title}")
    print("=" * 60)
    if 'accuracy_std' in metrics:
        print(f"Accuracy:    {metrics['accuracy']:.1f} ± {metrics['accuracy_std']:.1f}%")
        print(f"Sensitivity: {metrics['sensitivity']:.1f} ± {metrics['sensitivity_std']:.1f}%")
        print(f"Specificity: {metrics['specificity']:.1f} ± {metrics['specificity_std']:.1f}%")
    else:
        print(f"Accuracy:    {metrics['accuracy']:.1f}%")
        print(f"Sensitivity: {metrics['sensitivity']:.1f}%")
        print(f"Specificity: {metrics['specificity']:.1f}%")

# Experiment I: Base Random Forest Model

Test RF with:
- 365 trees
- 1 feature at each split
- Separately test linear and nonlinear variables
- Run 10 times with different seeds to compute standard error

In [ ]:
print("EXPERIMENT I: Base Random Forest Model")
print("=" * 70)

# Prepare data for linear variables
X_train_linear = train_df[linear_variables].values
X_test_linear = test_df[linear_variables].values
y_train = train_df['Faller'].values
y_test = test_df['Faller'].values

# Standardize
scaler_linear = StandardScaler()
X_train_linear_scaled = scaler_linear.fit_transform(X_train_linear)
X_test_linear_scaled = scaler_linear.transform(X_test_linear)

# Train with linear variables (10 runs)
print("\nTraining with LINEAR variables (10 runs)...")
preds_linear, models_linear = train_rf_model(
    X_train_linear_scaled, y_train, X_test_linear_scaled, 
    n_trees=365, max_features=1, n_runs=10
)
metrics_linear = calculate_metrics(y_test, preds_linear, n_runs=10)
print_metrics(metrics_linear, "Results with Linear Variables")

In [ ]:
# Prepare data for nonlinear variables
X_train_nonlinear = train_df[nonlinear_variables].values
X_test_nonlinear = test_df[nonlinear_variables].values

# Standardize
scaler_nonlinear = StandardScaler()
X_train_nonlinear_scaled = scaler_nonlinear.fit_transform(X_train_nonlinear)
X_test_nonlinear_scaled = scaler_nonlinear.transform(X_test_nonlinear)

# Train with nonlinear variables (10 runs)
print("\nTraining with NONLINEAR variables (10 runs)...")
preds_nonlinear, models_nonlinear = train_rf_model(
    X_train_nonlinear_scaled, y_train, X_test_nonlinear_scaled,
    n_trees=365, max_features=1, n_runs=10
)
metrics_nonlinear = calculate_metrics(y_test, preds_nonlinear, n_runs=10)
print_metrics(metrics_nonlinear, "Results with Nonlinear Variables")

# Experiment II: Feature Engineering with PCA

Steps:
1. Apply unsupervised feature selection (variance threshold)
2. Apply PCA to extract principal components (99% variability)
3. Train RF models on PCs

In [ ]:
print("\n\nEXPERIMENT II: Feature Engineering with PCA")
print("=" * 70)

# Step 1: Unsupervised feature selection (variance threshold)
# Remove low-variance features
variance_threshold = 0.01  # You may need to adjust this

# Linear variables
selector_linear = VarianceThreshold(threshold=variance_threshold)
X_train_linear_selected = selector_linear.fit_transform(X_train_linear_scaled)
X_test_linear_selected = selector_linear.transform(X_test_linear_scaled)

# Nonlinear variables
selector_nonlinear = VarianceThreshold(threshold=variance_threshold)
X_train_nonlinear_selected = selector_nonlinear.fit_transform(X_train_nonlinear_scaled)
X_test_nonlinear_selected = selector_nonlinear.transform(X_test_nonlinear_scaled)

n_linear_selected = X_train_linear_selected.shape[1]
n_nonlinear_selected = X_train_nonlinear_selected.shape[1]

print(f"\nAfter variance threshold:")
print(f"Linear variables: {len(linear_variables)} -> {n_linear_selected}")
print(f"Nonlinear variables: {len(nonlinear_variables)} -> {n_nonlinear_selected}")
print(f"Total features after selection: {n_linear_selected + n_nonlinear_selected}")

In [ ]:
# Step 2: Apply PCA with 99% variance explained

# PCA on linear variables
pca_linear = PCA(n_components=0.99, random_state=42)
X_train_linear_pca = pca_linear.fit_transform(X_train_linear_selected)
X_test_linear_pca = pca_linear.transform(X_test_linear_selected)
n_pcs_linear = X_train_linear_pca.shape[1]

# PCA on nonlinear variables
pca_nonlinear = PCA(n_components=0.99, random_state=42)
X_train_nonlinear_pca = pca_nonlinear.fit_transform(X_train_nonlinear_selected)
X_test_nonlinear_pca = pca_nonlinear.transform(X_test_nonlinear_selected)
n_pcs_nonlinear = X_train_nonlinear_pca.shape[1]

print(f"\nPCA Results (99% variance):")
print(f"Linear PCs: {n_pcs_linear}")
print(f"Nonlinear PCs: {n_pcs_nonlinear}")
print(f"Linear variance explained: {pca_linear.explained_variance_ratio_.sum():.4f}")
print(f"Nonlinear variance explained: {pca_nonlinear.explained_variance_ratio_.sum():.4f}")

In [ ]:
# Step 3: Train RF models on PCs

# Linear PCs (10 runs)
print("\nTraining with Linear PCs (10 runs)...")
preds_linear_pca, models_linear_pca = train_rf_model(
    X_train_linear_pca, y_train, X_test_linear_pca,
    n_trees=365, max_features=1, n_runs=10
)
metrics_linear_pca = calculate_metrics(y_test, preds_linear_pca, n_runs=10)
print_metrics(metrics_linear_pca, f"Results with {n_pcs_linear} Linear PCs")

# Nonlinear PCs (10 runs)
print("\nTraining with Nonlinear PCs (10 runs)...")
preds_nonlinear_pca, models_nonlinear_pca = train_rf_model(
    X_train_nonlinear_pca, y_train, X_test_nonlinear_pca,
    n_trees=365, max_features=1, n_runs=10
)
metrics_nonlinear_pca = calculate_metrics(y_test, preds_nonlinear_pca, n_runs=10)
print_metrics(metrics_nonlinear_pca, f"Results with {n_pcs_nonlinear} Nonlinear PCs")

# Experiment III: Combined Model with Elbow Point Analysis

Build upon nonlinear PCs and gradually add linear PCs to find the optimal combination.

In [ ]:
print("\n\nEXPERIMENT III: Combined Model with Elbow Point Analysis")
print("=" * 70)

# Store results for different numbers of linear PCs
results = []
max_linear_pcs = n_pcs_linear

print(f"\nTesting combinations: {n_pcs_nonlinear} nonlinear PCs + 0 to {max_linear_pcs} linear PCs")
print("This may take a few minutes...\n")

for n_linear in range(max_linear_pcs + 1):
    if n_linear == 0:
        # Just nonlinear PCs
        X_train_combined = X_train_nonlinear_pca
        X_test_combined = X_test_nonlinear_pca
    else:
        # Combine nonlinear PCs with first n linear PCs
        X_train_combined = np.hstack([
            X_train_nonlinear_pca,
            X_train_linear_pca[:, :n_linear]
        ])
        X_test_combined = np.hstack([
            X_test_nonlinear_pca,
            X_test_linear_pca[:, :n_linear]
        ])
    
    # Train model (10 runs)
    preds_combined, models_combined = train_rf_model(
        X_train_combined, y_train, X_test_combined,
        n_trees=365, max_features=1, n_runs=10
    )
    
    # Calculate metrics
    metrics = calculate_metrics(y_test, preds_combined, n_runs=10)
    
    # Calculate OOB score (use first model from the 10 runs)
    # Note: We need to retrain with oob_score=True
    rf_oob = RandomForestClassifier(
        n_estimators=365, max_features=1, random_state=42, 
        oob_score=True, n_jobs=-1
    )
    rf_oob.fit(X_train_combined, y_train)
    oob_score = rf_oob.oob_score_
    
    # Calculate AUC (use predictions from first run)
    y_proba = models_combined[0].predict_proba(X_test_combined)[:, 1]
    auc_score = roc_auc_score(y_test, y_proba)
    
    results.append({
        'n_linear_pcs': n_linear,
        'n_total_pcs': n_pcs_nonlinear + n_linear,
        'accuracy': metrics['accuracy'],
        'accuracy_std': metrics['accuracy_std'],
        'sensitivity': metrics['sensitivity'],
        'sensitivity_std': metrics['sensitivity_std'],
        'specificity': metrics['specificity'],
        'specificity_std': metrics['specificity_std'],
        'oob_score': oob_score * 100,
        'auc': auc_score
    })
    
    print(f"Linear PCs: {n_linear:2d} | Total PCs: {n_pcs_nonlinear + n_linear:2d} | "
          f"Acc: {metrics['accuracy']:.1f}±{metrics['accuracy_std']:.1f}% | "
          f"Sens: {metrics['sensitivity']:.1f}±{metrics['sensitivity_std']:.1f}% | "
          f"Spec: {metrics['specificity']:.1f}±{metrics['specificity_std']:.1f}% | "
          f"OOB: {oob_score*100:.1f}% | AUC: {auc_score:.3f}")

# Convert to DataFrame
results_df = pd.DataFrame(results)
print("\nCompleted all combinations!")

In [ ]:
# Plot elbow curves
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot 1: OOB Score
axes[0].plot(results_df['n_linear_pcs'], results_df['oob_score'], 'o-', linewidth=2, markersize=8)
axes[0].set_xlabel('Number of Linear PCs Added', fontsize=12)
axes[0].set_ylabel('OOB Score (%)', fontsize=12)
axes[0].set_title('(a) Out-of-Bag Score vs Number of Linear PCs', fontsize=13, fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].axvline(x=4, color='red', linestyle='--', alpha=0.7, label='Elbow point (n=4)')
axes[0].legend()

# Plot 2: AUC Score
axes[1].plot(results_df['n_linear_pcs'], results_df['auc'], 'o-', linewidth=2, markersize=8, color='orange')
axes[1].set_xlabel('Number of Linear PCs Added', fontsize=12)
axes[1].set_ylabel('AUC Score', fontsize=12)
axes[1].set_title('(b) AUC Score vs Number of Linear PCs', fontsize=13, fontweight='bold')
axes[1].grid(True, alpha=0.3)
axes[1].axvline(x=4, color='red', linestyle='--', alpha=0.7, label='Elbow point (n=4)')
axes[1].legend()

plt.tight_layout()
plt.savefig('../scripts/experiment_III_elbow_plot.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nElbow plot saved as 'experiment_III_elbow_plot.png'")

In [ ]:
# Identify elbow point (we'll use n=4 as mentioned in the paper)
elbow_point = 4
best_result = results_df[results_df['n_linear_pcs'] == elbow_point].iloc[0]

print(f"\nBest Model (Elbow Point at {elbow_point} Linear PCs):")
print("=" * 60)
print(f"Total PCs: {best_result['n_total_pcs']} ({n_pcs_nonlinear} nonlinear + {elbow_point} linear)")
print(f"Accuracy:    {best_result['accuracy']:.1f} ± {best_result['accuracy_std']:.1f}%")
print(f"Sensitivity: {best_result['sensitivity']:.1f} ± {best_result['sensitivity_std']:.1f}%")
print(f"Specificity: {best_result['specificity']:.1f} ± {best_result['specificity_std']:.1f}%")
print(f"OOB Score:   {best_result['oob_score']:.1f}%")
print(f"AUC Score:   {best_result['auc']:.3f}")

# Summary Table: Comparison of All Experiments

In [ ]:
# Create comprehensive summary table
summary_data = [
    {
        'Experiment': 'I: Linear Variables',
        'Features': f'{len(linear_variables)} linear vars',
        'Accuracy': f"{metrics_linear['accuracy']:.1f} ± {metrics_linear['accuracy_std']:.1f}",
        'Sensitivity': f"{metrics_linear['sensitivity']:.1f} ± {metrics_linear['sensitivity_std']:.1f}",
        'Specificity': f"{metrics_linear['specificity']:.1f} ± {metrics_linear['specificity_std']:.1f}"
    },
    {
        'Experiment': 'I: Nonlinear Variables',
        'Features': f'{len(nonlinear_variables)} nonlinear vars',
        'Accuracy': f"{metrics_nonlinear['accuracy']:.1f} ± {metrics_nonlinear['accuracy_std']:.1f}",
        'Sensitivity': f"{metrics_nonlinear['sensitivity']:.1f} ± {metrics_nonlinear['sensitivity_std']:.1f}",
        'Specificity': f"{metrics_nonlinear['specificity']:.1f} ± {metrics_nonlinear['specificity_std']:.1f}"
    },
    {
        'Experiment': 'II: Linear PCs',
        'Features': f'{n_pcs_linear} PCs',
        'Accuracy': f"{metrics_linear_pca['accuracy']:.1f} ± {metrics_linear_pca['accuracy_std']:.1f}",
        'Sensitivity': f"{metrics_linear_pca['sensitivity']:.1f} ± {metrics_linear_pca['sensitivity_std']:.1f}",
        'Specificity': f"{metrics_linear_pca['specificity']:.1f} ± {metrics_linear_pca['specificity_std']:.1f}"
    },
    {
        'Experiment': 'II: Nonlinear PCs',
        'Features': f'{n_pcs_nonlinear} PCs',
        'Accuracy': f"{metrics_nonlinear_pca['accuracy']:.1f} ± {metrics_nonlinear_pca['accuracy_std']:.1f}",
        'Sensitivity': f"{metrics_nonlinear_pca['sensitivity']:.1f} ± {metrics_nonlinear_pca['sensitivity_std']:.1f}",
        'Specificity': f"{metrics_nonlinear_pca['specificity']:.1f} ± {metrics_nonlinear_pca['specificity_std']:.1f}"
    },
    {
        'Experiment': 'III: Combined (Best)',
        'Features': f'{best_result["n_total_pcs"]:.0f} PCs ({n_pcs_nonlinear} NL + {elbow_point} L)',
        'Accuracy': f"{best_result['accuracy']:.1f} ± {best_result['accuracy_std']:.1f}",
        'Sensitivity': f"{best_result['sensitivity']:.1f} ± {best_result['sensitivity_std']:.1f}",
        'Specificity': f"{best_result['specificity']:.1f} ± {best_result['specificity_std']:.1f}"
    }
]

summary_df = pd.DataFrame(summary_data)
print("\n" + "=" * 100)
print("SUMMARY: All Experiments")
print("=" * 100)
print(summary_df.to_string(index=False))
print("=" * 100)

# Save summary to CSV
summary_df.to_csv('../scripts/experiments_summary.csv', index=False)
print("\nSummary saved to 'experiments_summary.csv'")

# Visualize Performance Comparison

In [ ]:
# Create bar plot comparing all experiments
fig, ax = plt.subplots(figsize=(14, 6))

experiments = ['Exp I\nLinear', 'Exp I\nNonlinear', 'Exp II\nLinear PCs', 
               'Exp II\nNonlinear PCs', 'Exp III\nCombined']

accuracy_vals = [
    metrics_linear['accuracy'],
    metrics_nonlinear['accuracy'],
    metrics_linear_pca['accuracy'],
    metrics_nonlinear_pca['accuracy'],
    best_result['accuracy']
]

sensitivity_vals = [
    metrics_linear['sensitivity'],
    metrics_nonlinear['sensitivity'],
    metrics_linear_pca['sensitivity'],
    metrics_nonlinear_pca['sensitivity'],
    best_result['sensitivity']
]

specificity_vals = [
    metrics_linear['specificity'],
    metrics_nonlinear['specificity'],
    metrics_linear_pca['specificity'],
    metrics_nonlinear_pca['specificity'],
    best_result['specificity']
]

x = np.arange(len(experiments))
width = 0.25

bars1 = ax.bar(x - width, accuracy_vals, width, label='Accuracy', color='steelblue')
bars2 = ax.bar(x, sensitivity_vals, width, label='Sensitivity', color='coral')
bars3 = ax.bar(x + width, specificity_vals, width, label='Specificity', color='lightgreen')

ax.set_ylabel('Score (%)', fontsize=12)
ax.set_title('Performance Comparison Across All Experiments', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(experiments, fontsize=10)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3, axis='y')
ax.set_ylim([0, 100])

# Add value labels on bars
for bars in [bars1, bars2, bars3]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.1f}', ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.savefig('../scripts/experiments_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("Comparison plot saved as 'experiments_comparison.png'")

# ROC Curve for Best Model

In [ ]:
# Train final best model and plot ROC curve
X_train_best = np.hstack([
    X_train_nonlinear_pca,
    X_train_linear_pca[:, :elbow_point]
])
X_test_best = np.hstack([
    X_test_nonlinear_pca,
    X_test_linear_pca[:, :elbow_point]
])

rf_best = RandomForestClassifier(
    n_estimators=365, max_features=1, random_state=42, n_jobs=-1
)
rf_best.fit(X_train_best, y_train)
y_proba_best = rf_best.predict_proba(X_test_best)[:, 1]

# Calculate ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_proba_best)
roc_auc = roc_auc_score(y_test, y_proba_best)

# Plot ROC curve
plt.figure(figsize=(8, 8))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.3f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curve - Best Model (Experiment III)', fontsize=14, fontweight='bold')
plt.legend(loc="lower right", fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../scripts/roc_curve_best_model.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"ROC AUC: {roc_auc:.3f}")
print("ROC curve saved as 'roc_curve_best_model.png'")

# Conclusion

This notebook successfully implemented all three experiments:

1. **Experiment I** tested base RF models with linear and nonlinear variables separately
2. **Experiment II** applied feature engineering with variance selection and PCA
3. **Experiment III** combined nonlinear and linear PCs, identifying the optimal combination

The best model uses a combination of nonlinear and linear principal components, achieving improved performance across all metrics.